In [2]:
import pandas as pd
import numpy as np
from itertools import product
from datetime import datetime,date, timedelta

# Process User Logs

In [22]:
user_logs = pd.read_csv("D:\\Northeastern\\50 pct undersample final data\\X_val_user_logs.csv",usecols=[1,2,3,4,5,6,7,8,9],nrows=1000)
user_logs.loc[:,["date"]]=pd.to_datetime(user_logs.date).dt.date
members_df = pd.read_csv("D:\\Northeastern\\50 pct undersample final data\\X_val_transformed.csv")

In [23]:
users = list(set(user_logs.msno))
num_users = len(users)
start_date = date(2016,1,1)#min(user_logs.date)
end_date = date(2017,1,31)#max(user_logs.date)

# this creates a list containing all dates from start to end
dates = [start_date + timedelta(days=x) for x in range((end_date-start_date).days + 1)]
num_dates=len(dates)

padded_df = pd.DataFrame(product(users, dates), columns=["msno","date"])
padded_df = padded_df.merge(user_logs, how='left', on=["msno","date"]).fillna(0)
padded_df = padded_df.sort_values(by=["msno","date"])

## Reshape User Logs and Save

In [24]:
# Reshape df to numpy 3d matrix
padded_df_data = padded_df.iloc[:,2:9]
#padded_df_data.to_csv("padded_output.csv")
num_cols = len(padded_df_data.columns)
padded_array = padded_df_data.values.reshape(num_users,num_dates,num_cols)
np.save("..\\Model Building\\val_padded_user_logs",padded_array)
padded_array.shape

(920, 397, 7)

# Save Members Array

In [21]:
# Grab users from members CSV
members_df_filtered=members_df[members_df["msno"].isin(users)].sort_values("msno")
members_df_data = members_df_filtered.iloc[:,1:]
np.save("..\\Model Building\\val_members",members_df_data.values)

# Save Labels Array

In [19]:
labels = pd.read_csv("D:\\Northeastern\\50 pct undersample final data\\y_val_with_msno.csv")
labels_filtered = labels[labels["msno"].isin(users)]
labels_filtered = labels_filtered.sort_values("msno")
labels_filtered = labels_filtered.drop(["msno"], axis=1)
np.save("..\\Model Building\\val_labels", labels_filtered.values)
labels_filtered.shape

(920, 1)